# model_parasm.py の使い方

http://nupic.docs.numenta.org/1.0.1/guides/swarming/running.html
にある例で、

$NUPIC/scripts/run_swarm.py $NUPIC/examples/swarm/simple/search_def.json  --maxWorkers=4
を実行すると。

examples/swarm/simpleに、model_0/model_params.pyができます。
その中身は、以下の通りです。

In [ ]:
# ----------------------------------------------------------------------
# Numenta Platform for Intelligent Computing (NuPIC)
# Copyright (C) 2013, Numenta, Inc.  Unless you have an agreement
# with Numenta, Inc., for a separate license for this software code, the
# following terms and conditions apply:
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero Public License version 3 as
# published by the Free Software Foundation.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Affero Public License for more details.
#
# You should have received a copy of the GNU Affero Public License
# along with this program.  If not, see http://www.gnu.org/licenses.
#
# http://numenta.org/licenses/
# ----------------------------------------------------------------------

MODEL_PARAMS = {'aggregationInfo': {'days': 0,
                     'fields': [(u'timestamp', 'first'),
                                (u'gym', 'first'),
                                (u'consumption', 'sum')],
                     'hours': 1,
                     'microseconds': 0,
                     'milliseconds': 0,
                     'minutes': 0,
                     'months': 0,
                     'seconds': 0,
                     'weeks': 0,
                     'years': 0},
 'model': 'HTMPrediction',
 'modelParams': {'anomalyParams': {u'anomalyCacheRecords': None,
                                   u'autoDetectThreshold': None,
                                   u'autoDetectWaitRecords': None},
                 'clParams': {'alpha': 0.050050000000000004,
                              'regionName': 'SDRClassifierRegion',
                              'steps': '1',
                              'verbosity': 0},
                 'inferenceType': 'NontemporalMultiStep',
                 'sensorParams': {'encoders': {'_classifierInput': {'classifierOnly': True,
                                                                    'clipInput': True,
                                                                    'fieldname': 'consumption',
                                                                    'n': 275,
                                                                    'name': '_classifierInput',
                                                                    'type': 'AdaptiveScalarEncoder',
                                                                    'w': 21},
                                               u'consumption': {'clipInput': True,
                                                                'fieldname': 'consumption',
                                                                'n': 272,
                                                                'name': 'consumption',
                                                                'type': 'AdaptiveScalarEncoder',
                                                                'w': 21},
                                               u'timestamp_dayOfWeek': None,
                                               u'timestamp_timeOfDay': None,
                                               u'timestamp_weekend': None},
                                  'sensorAutoReset': None,
                                  'verbosity': 0},
                 'spEnable': True,
                 'spParams': {'boostStrength': 0.0,
                              'columnCount': 2048,
                              'globalInhibition': 1,
                              'inputWidth': 0,
                              'numActiveColumnsPerInhArea': 40,
                              'potentialPct': 0.8,
                              'seed': 1956,
                              'spVerbosity': 0,
                              'spatialImp': 'cpp',
                              'synPermActiveInc': 0.05,
                              'synPermConnected': 0.1,
                              'synPermInactiveDec': 0.05015},
                 'tmEnable': True,
                 'tmParams': {'activationThreshold': 14,
                              'cellsPerColumn': 32,
                              'columnCount': 2048,
                              'globalDecay': 0.0,
                              'initialPerm': 0.21,
                              'inputWidth': 2048,
                              'maxAge': 0,
                              'maxSegmentsPerCell': 128,
                              'maxSynapsesPerSegment': 32,
                              'minThreshold': 11,
                              'newSynapseCount': 20,
                              'outputType': 'normal',
                              'pamLength': 3,
                              'permanenceDec': 0.1,
                              'permanenceInc': 0.1,
                              'seed': 1960,
                              'temporalImp': 'cpp',
                              'verbosity': 0},
                 'trainSPNetOnlyIfRequested': False},
 'predictAheadTime': None,
 'version': 1}

以下のAPIアルゴリズムのプログラムで読み込まれるパラメータに相当していますので、下のプログラムを、上のmodel_params.pyから読む形に変更するということになります。  
model_params.py は、辞書型の変数が定義されているだけですので、プログラムと同じディレクトリに、このmodel_params.pyを置いて、プログラムで、from  model_params import MODEL_PARAMS とやれば、プログラム内で、辞書型の変数MODEL_PARAMSが参照できるようになります。  

http://nupic.docs.numenta.org/1.0.1/quick-start/algorithms.html

In [ ]:
def runHotgym(numRecords):
    with open(_PARAMS_PATH, "r") as f:
        modelParams = yaml.safe_load(f)["modelParams"]
        enParams = modelParams["sensorParams"]["encoders"]
        spParams = modelParams["spParams"]
        tmParams = modelParams["tmParams"]

    timeOfDayEncoder = DateEncoder(
    timeOfDay=enParams["timestamp_timeOfDay"]["timeOfDay"])
    weekendEncoder = DateEncoder(weekend=enParams["timestamp_weekend"]["weekend"])
    scalarEncoder = RandomDistributedScalarEncoder(enParams["consumption"]["resolution"])

    encodingWidth = (timeOfDayEncoder.getWidth()
                   + weekendEncoder.getWidth()
                   + scalarEncoder.getWidth())

sp = SpatialPooler(
    inputDimensions=(encodingWidth,),
    columnDimensions=(spParams["columnCount"],),
    potentialPct=spParams["potentialPct"],
    potentialRadius=encodingWidth,
    globalInhibition=spParams["globalInhibition"],
    localAreaDensity=spParams["localAreaDensity"],
    numActiveColumnsPerInhArea=spParams["numActiveColumnsPerInhArea"],
    synPermInactiveDec=spParams["synPermInactiveDec"],
    synPermActiveInc=spParams["synPermActiveInc"],
    synPermConnected=spParams["synPermConnected"],
    boostStrength=spParams["boostStrength"],
    seed=spParams["seed"],
    wrapAround=True
  )

tm = TemporalMemory(
    columnDimensions=(tmParams["columnCount"],),
    cellsPerColumn=tmParams["cellsPerColumn"],
    activationThreshold=tmParams["activationThreshold"],
    initialPermanence=tmParams["initialPerm"],
    connectedPermanence=spParams["synPermConnected"],
    minThreshold=tmParams["minThreshold"],
    maxNewSynapseCount=tmParams["newSynapseCount"],
    permanenceIncrement=tmParams["permanenceInc"],
    permanenceDecrement=tmParams["permanenceDec"],
    predictedSegmentDecrement=0.0,
    maxSegmentsPerCell=tmParams["maxSegmentsPerCell"],
    maxSynapsesPerSegment=tmParams["maxSynapsesPerSegment"],
    seed=tmParams["seed"]
  )

# パラメータの読み出し例

In [ ]:
from model_params import MODEL_PARAMS

#読み出し例
MODEL_PARAMS['aggregationInfo']['days']


上のプログラムでは、下のように、yaml からmodelParams を読んでいます。

In [ ]:
with open(_PARAMS_PATH, "r") as f:
    modelParams = yaml.safe_load(f)["modelParams"]
    enParams = modelParams["sensorParams"]["encoders"]
    spParams = modelParams["spParams"]
    tmParams = modelParams["tmParams"]

そうではなしに、model_params.py から MODEL_PARAMS をインポートします。

In [5]:
from model_params import MODEL_PARAMS

enParams = MODEL_PARAMS['modelParams']["sensorParams"]["encoders"]
spParams = MODEL_PARAMS['modelParams']["spParams"]
tmParams = MODEL_PARAMS['modelParams']["tmParams"]
    
print enParams
print spParams
print tmParams


{'_classifierInput': {'classifierOnly': True, 'name': '_classifierInput', 'clipInput': True, 'n': 275, 'fieldname': 'consumption', 'w': 21, 'type': 'AdaptiveScalarEncoder'}, u'timestamp_weekend': None, u'timestamp_timeOfDay': None, u'consumption': {'name': 'consumption', 'clipInput': True, 'n': 272, 'fieldname': 'consumption', 'w': 21, 'type': 'AdaptiveScalarEncoder'}, u'timestamp_dayOfWeek': None}
{'columnCount': 2048, 'synPermInactiveDec': 0.05015, 'spatialImp': 'cpp', 'synPermConnected': 0.1, 'seed': 1956, 'numActiveColumnsPerInhArea': 40, 'globalInhibition': 1, 'inputWidth': 0, 'spVerbosity': 0, 'synPermActiveInc': 0.05, 'potentialPct': 0.8, 'boostStrength': 0.0}
{'columnCount': 2048, 'activationThreshold': 14, 'pamLength': 3, 'cellsPerColumn': 32, 'permanenceDec': 0.1, 'minThreshold': 11, 'inputWidth': 2048, 'maxSynapsesPerSegment': 32, 'outputType': 'normal', 'initialPerm': 0.21, 'globalDecay': 0.0, 'maxAge': 0, 'newSynapseCount': 20, 'maxSegmentsPerCell': 128, 'permanenceInc': 0